In [24]:
import openpyxl
import re

# Load Excel template
template_path = "KC 2025-26 Applications KPI_TEMPLATE.xlsx"
wb = openpyxl.load_workbook(template_path)

# Define your values
variable_Extraction_date = "May 16, 2025" #Enter date of data extraction. Format: "Month DD, YYYY"
variable_Current_enrolment_academic_year = "2025-26" #Enter the current academic year. Format: "YYYY-YY"

variable_map = {
    "Test_1": 0.3,
    "Test_2": 57
}

# Regular expression to detect {{variable}} patterns
pattern = re.compile(r"{{(.*?)}}")

# Loop through all worksheets and replace variables
for ws in wb.worksheets:
    for row in ws.iter_rows():
        for cell in row:
            if isinstance(cell.value, str):
                matches = pattern.findall(cell.value)
                for match in matches:
                    if match in variable_map:
                        cell.value = cell.value.replace(f"{{{{{match}}}}}", str(variable_map[match]))

# Save to new file
output_filename = f"KC {variable_Current_enrolment_academic_year} Applications KPI as of {variable_Extraction_date}.xlsx"
wb.save(output_filename)


In [21]:
from openpyxl import load_workbook
from openpyxl.formatting.rule import IconSetRule

# Load workbook and worksheet
template_path = "KC 2025-26 Applications KPI_TEMPLATE.xlsx"
wb = load_workbook(output_filename)
ws = wb["Summer 2025 Apps By School"]

# Define the range
target_range = "P2:P500"

# Apply icon rule: green (≥1), yellow (0.6–1), red (<0.6)
icon_rule = IconSetRule(
    '3Signs',       # Icon set with green check, yellow exclamation, red cross
    'num',          # Compare numeric values
    [0, 0.6, 1],       # Thresholds
    showValue=True,
    percent=False,
    reverse=False   # Don't reverse the icon order
)

# Apply to target range
ws.conditional_formatting.add(target_range, icon_rule)

# Save workbook
output_path = "Formatted_Output.xlsx"
wb.save(output_path)


In [2]:
import openpyxl
import re

# Load Excel template
template_path = "KC 2025-26 Applications KPI_TEMPLATE.xlsx"
wb = openpyxl.load_workbook(template_path)

# Define your values
variable_Extraction_date = "May 16, 2025"  # Format: "Month DD, YYYY"
variable_Current_enrolment_academic_year = "2025-26"  # Format: "YYYY-YY"
variable_academic_year = "2025"

variable_map = {
    "Test_1": 0.3432453252345,
    "Test_2": 57,
    "SuYr": variable_academic_year,
    "FYr": variable_academic_year,
    "WYr": variable_academic_year,
    "SpYr": variable_academic_year
}

# Regular expression to detect {{variable}} patterns
pattern = re.compile(r"{{(.*?)}}")

# Loop through all worksheets and replace variables
for ws in wb.worksheets:
    for row in ws.iter_rows():
        for cell in row:
            if isinstance(cell.value, str):
                matches = pattern.findall(cell.value)
                for match in matches:
                    if match in variable_map:
                        cell.value = cell.value.replace(f"{{{{{match}}}}}", str(variable_map[match]))

# Rename sheets by replacing placeholders in their names
for ws in wb.worksheets:
    matches = pattern.findall(ws.title)
    new_title = ws.title
    for match in matches:
        if match in variable_map:
            new_title = new_title.replace(f"{{{{{match}}}}}", str(variable_map[match]))
    if new_title != ws.title:
        ws.title = new_title

# Convert text to numbers in column P for specific sheets
target_sheets = [f"Summer {variable_academic_year} Apps By School", f"Fall {variable_academic_year} Apps By School", f"Winter {variable_academic_year} Apps By School", f"Spring {variable_academic_year} Apps By School"]
column_p_index = openpyxl.utils.column_index_from_string('P')  # Column P = 16

for sheet_name in target_sheets:
    if sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        for row in ws.iter_rows(min_col=column_p_index, max_col=column_p_index):
            cell = row[0]
            if isinstance(cell.value, str) and cell.value.strip().replace(',', '').replace('.', '', 1).isdigit():
                try:
                    cell.value = float(cell.value.replace(',', '')) if '.' in cell.value else int(cell.value.replace(',', ''))
                except ValueError:
                    pass  # Skip if conversion fails

# Save to new file
output_filename = f"KC {variable_Current_enrolment_academic_year} Applications KPI as of {variable_Extraction_date}.xlsx"
wb.save(output_filename)


In [4]:
#Import Modules

import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import column_index_from_string
import re

#Function for dividing numerator by denominator, calculating a decimal value. Return None if denominator is None. Treat numerator as 0 when it is None. 
def division_calculator_decimal(numerator, denominator): 
    if denominator in [None, 0]:
        return None
    if numerator is None:
        numerator = 0
    try:
        return numerator / denominator
    except TypeError:
        return None

#Load files. The file names must match exactly the names of the application data files. Manual Input Required.

term_1_file_application_comparative = "Application Comparative-Summer 2025 as of May 16, 2025.xlsx"
term_1_file_application_details = "Applications Reports Details-IR-New-Summer 2025 as of May 01, 2025 (Fake Data).xlsx"

term_2_file_application_comparative = "Application Comparative-Fall 2025 as of May 16, 2025.xlsx"
term_2_file_application_details = "Applications Reports Details-IR-New-Fall 2025 as of May 01, 2025 (Fake Data).xlsx"

term_3_file_application_comparative = "Application Comparative-Winter 2026 as of May 16, 2025.xlsx"
term_3_file_application_details = "Applications Reports Details-IR-New-Winter 2026 as of May 01, 2025 (Fake Data).xlsx"

term_4_file_application_comparative = "Application Comparative-Spring 2026 as of May 16, 2025.xlsx"
term_4_file_application_details = "Applications Reports Details-IR-New-Spring 2026 as of May 01, 2025 (Fake Data).xlsx"

#Manually define variables
variable_Extraction_date = "May 25, 2025" #Enter date of data extraction. Format: "Month DD, YYYY"
variable_range_applications_academic_year = "2025-26 & 2026-27" #Enter the academic year(s) of your application data. Format: "YYYY-YY" or "YYYY-YY & YYYY-YY"
variable_current_academic_year_file_name = "2025-26" #Enter academic year for output file name. Format: "YYYY-YY"

# Load the workbook
wb = openpyxl.load_workbook(term_1_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_1_details = pd.read_excel(term_1_file_application_details, sheet_name=matched_sheet_name)

program_ID_Business_Administration = [ 
    'BABUSC', 'BAACTD', 'BABUSD', 'BAESPM', 'BAHRMD', 'GSCMD', 'DIGMD',
    'BAATCO', 'BABSCO', 'NAO', 'ALO', 'HOSMNG', 'BAHRMC', 'OFAC'
]

populations = ['DOMESTIC', 'INTERNATIONAL']
years = ['1', '2']

# This dictionary will store all your generated variables
program_stats = {}

# Helper to avoid repetitive filtering
def base_filter(df, program, pop, year):
    return df[
        (df['CURRICULUM'].str.replace(' ','').str.upper() == program) &
        (df['Population'].str.replace(' ','').str.upper() == pop) &
        (df['Year'].astype(str).str.strip() == year)
    ]

# Loop through each program, population, and year
for program in program_ID_Business_Administration:
    for pop in populations:
        for year in years:
            key_base = f"term_1_{program}_{pop}_Year_{year}"

            df_filtered = base_filter(df_term_1_details, program, pop, year)
            
            program_stats[f"{key_base}_Applications"] = len(df_filtered) if len(df_filtered) > 0 else None
            
            program_stats[f"{key_base}_Accepted"] = len(df_filtered[
                (df_filtered['Accepted'].isin(['ACPT', 'ACLN', 'COND'])) &
                ~(df_filtered['Deferred Application'].isin(['ACPT', 'COND']))
            ]) if not df_filtered.empty else None

            program_stats[f"{key_base}_Payment_Pending"] = len(df_filtered[
                ~(df_filtered['Payment Pending'].isna() | (df_filtered['Payment Pending'] == ''))
            ]) if not df_filtered.empty else None

            program_stats[f"{key_base}_Denied"] = len(df_filtered[
                df_filtered['Denied'].isin(['APEX', 'DENY', 'NTSL'])
            ]) if not df_filtered.empty else None

            program_stats[f"{key_base}_Declined"] = len(df_filtered[
                df_filtered['Declined'].isin(['CCAN', 'DECL'])
            ]) if not df_filtered.empty else None

            program_stats[f"{key_base}_Pending"] = len(df_filtered[
                df_filtered['Pending'].isin(['APRO', 'NODE'])
            ]) if not df_filtered.empty else None

            program_stats[f"{key_base}_Waitlisted"] = len(df_filtered[
                df_filtered['Waitlisted'].isin(['CDMT', 'RQMT', 'WAIT'])
            ]) if not df_filtered.empty else None

            program_stats[f"{key_base}_Enrolled"] = len(df_filtered[
                df_filtered['Enrollment'].isin(['ENRO'])
            ]) if not df_filtered.empty else None

            # Conversion rate using your custom calculator
            enrolled = program_stats[f"{key_base}_Enrolled"]
            accepted = program_stats[f"{key_base}_Accepted"]
            program_stats[f"{key_base}_Conversion_Rate"] = division_calculator_decimal(enrolled, accepted)



In [9]:
# Get total applications for BABUSC Domestic Year 1
program_stats["term_1_BABUSC_DOMESTIC_Year_1_Applications"]

# Get all keys
all_keys = list(program_stats.keys())

# Filter keys by program and population if needed
babusc_domestic_keys = [k for k in all_keys if "BABUSC_DOMESTIC" in k]

In [12]:
all_keys

['term_1_BABUSC_DOMESTIC_Year_1_Applications',
 'term_1_BABUSC_DOMESTIC_Year_1_Accepted',
 'term_1_BABUSC_DOMESTIC_Year_1_Payment_Pending',
 'term_1_BABUSC_DOMESTIC_Year_1_Denied',
 'term_1_BABUSC_DOMESTIC_Year_1_Declined',
 'term_1_BABUSC_DOMESTIC_Year_1_Pending',
 'term_1_BABUSC_DOMESTIC_Year_1_Waitlisted',
 'term_1_BABUSC_DOMESTIC_Year_1_Enrolled',
 'term_1_BABUSC_DOMESTIC_Year_1_Conversion_Rate',
 'term_1_BABUSC_DOMESTIC_Year_2_Applications',
 'term_1_BABUSC_DOMESTIC_Year_2_Accepted',
 'term_1_BABUSC_DOMESTIC_Year_2_Payment_Pending',
 'term_1_BABUSC_DOMESTIC_Year_2_Denied',
 'term_1_BABUSC_DOMESTIC_Year_2_Declined',
 'term_1_BABUSC_DOMESTIC_Year_2_Pending',
 'term_1_BABUSC_DOMESTIC_Year_2_Waitlisted',
 'term_1_BABUSC_DOMESTIC_Year_2_Enrolled',
 'term_1_BABUSC_DOMESTIC_Year_2_Conversion_Rate',
 'term_1_BABUSC_INTERNATIONAL_Year_1_Applications',
 'term_1_BABUSC_INTERNATIONAL_Year_1_Accepted',
 'term_1_BABUSC_INTERNATIONAL_Year_1_Payment_Pending',
 'term_1_BABUSC_INTERNATIONAL_Year_1_